# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2023-04-19 03:52:10--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2023-04-19 03:52:10 (51.9 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [ ]:
data.drop(data[data['variety']=='Virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [ ]:
data['variety']=data['variety'].map({'Setosa':-1,'Versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

20
80


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [ ]:
def hitung_cost_gradient(W,X,Y,regularization):
  jarak = 1 - (Y*np.dot(X,W))
  dw = np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di = W - (regularization * Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [ ]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [ ]:
W = sgd(data_latih,label_latih)
print(W)

[-0.22559932 -0.7150439   1.13588124  0.51445469]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [ ]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [ ]:
y_prediksi = testing(W, data_uji)
print(sum(y_prediksi==label_uji))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [ ]:
data_tugas = pd.read_csv('iris.csv')
data_backup = data_tugas

data_tugas.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [ ]:
data_tugas.drop(data_tugas[data_tugas['variety']=='Setosa'].index,inplace=True)
data_tugas['variety']=data_tugas['variety'].map({'Virginica':1,'Versicolor':-1})

data_tugas.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
50,7.0,3.2,4.7,1.4,-1
51,6.4,3.2,4.5,1.5,-1
52,6.9,3.1,4.9,1.5,-1
53,5.5,2.3,4.0,1.3,-1
54,6.5,2.8,4.6,1.5,-1


In [ ]:
tugas_latih,tugas_uji = train_test_split(data_tugas,test_size=0.2)

print(tugas_uji.shape[0])
print(tugas_latih.shape[0])

20
80


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [ ]:
label_latih_t = tugas_latih.pop('variety')
label_uji_t = tugas_uji.pop('variety')

In [ ]:
W2 = sgd(tugas_latih,label_latih_t)
print(W2)

[-1.92697967 -4.89916093  3.63658215  5.60401552]


In [ ]:
tugas_prediksi = testing(W2, tugas_uji)
print(sum(tugas_prediksi==label_uji_t))

18


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi sgd. Apakah hasilnya lebih baik? Mengapa?

In [ ]:
def sgd_tugas(data_latih,label_latih,learning_rate = 0.000001,max_epoch=5000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

In [ ]:
W2 = sgd_tugas(tugas_latih,label_latih_t)
print(W2)

[-1.8309455  -5.51392914  3.77201526  5.9827183 ]


In [ ]:
tugas_prediksi = testing(W2, tugas_uji)
print(sum(tugas_prediksi==label_uji_t))

17


Apakah hasilnya lebih baik? Mengapa?
JAWAB
Hasilnya lebih baik karena mampu mengenali 18 dari 20 data uji. Epoch adalah hyperparameter yang menentukan berapa kali algoritma pembelajaran akan bekerja mengolah seluruh dataset training. Jadi, semakin tinggi nilai epoch maka bisa jadi semakin tinggi juga akurasi dari model. Namun, terdapat batasan dimana hasil tidak akan berubah lagi apabila kita terus menambah epoch.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [ ]:
df = pd.read_csv('iris.csv')

df.drop("petal.width",axis=1,inplace=True)
df.drop("petal.length",axis=1,inplace=True)

df.head()

,sepal.length,sepal.width,variety
0,5.1,3.5,Setosa
1,4.9,3.0,Setosa
2,4.7,3.2,Setosa
3,4.6,3.1,Setosa
4,5.0,3.6,Setosa


In [ ]:
df.drop(df[df['variety']=='Setosa'].index,inplace=True)
df['variety']=df['variety'].map({'Virginica':1,'Versicolor':-1})

df_latih,df_uji = train_test_split(data_tugas,test_size=0.2)

label_latih_df = df_latih.pop('variety')
label_uji_df = df_uji.pop('variety')

In [ ]:
Wdf = sgd(df_latih,label_latih_df)
print(Wdf)

[-4.58337837 -3.51745458  6.87169478  4.06637471]


In [ ]:
df_prediksi = testing(Wdf, df_uji)
print(sum(df_prediksi==label_uji_df))

18


Apakah hasilnya lebih baik?
JAWAB
Tidak, karena data yang digunakan untuk pembelajaran/training semakin sedikit.